# Week 5 Solution (tentative)

In [1]:
\l C:\q\columbiaHdb\

In [2]:
/ print all variables
\v

`bin10`date


In [3]:
/ set console size
\c 30 100

In [4]:
tbl: select from bin10 where date = 2019.01.03

In [21]:
/ note that some dates have questionable data, you can drop the date 2019.01.09 for future homework
`x xasc select count i by date from bin10 

date      | x      
----------| -------
2019.01.09| 29058  
2019.12.24| 610324 
2019.11.29| 616629 
2019.07.03| 626717 
2019.09.18| 887893 
2019.03.08| 1122500
2019.12.10| 1133044
2019.12.11| 1133044
2019.12.12| 1133044
2019.12.13| 1133044
2019.12.16| 1133044
2019.12.17| 1133044
2019.12.18| 1133044
2019.12.19| 1133044
2019.12.20| 1133044
2019.12.23| 1133044
2019.12.26| 1133044
2019.12.27| 1133044
2019.12.09| 1135385
2019.12.06| 1139580
2019.12.05| 1140067
2019.11.21| 1144749
2019.11.22| 1144749
2019.11.25| 1144749
2019.11.26| 1144749
..


# OW Model
The baseline OW model proposes that
$$
    I_{t + \delta t} - I_t = - \beta I_t \delta t + \lambda \sigma \frac{q_t}{\text{ADV}},
$$
or equivalently,
$$
    I_{t + \delta t} = (1 - \beta \delta t) I_t + \lambda \sigma \frac{q_t}{\text{ADV}}.
$$
This is an exponentially weighted average (EMA) on $$ \color{red}{\frac{1}{\beta \delta t}} \lambda \sigma \frac{q_t}{\text{ADV}}$$ with smoothing factor $\beta \delta t$.

# Data Cleaning

Some id has missing values at the beginning.

In [6]:
select from tbl where id = 2

date       time     id trade     mid      spread       vol          adv         
--------------------------------------------------------------------------------
           09:30:00 2  0                                                        
           09:30:10 2  0                                                        
           09:30:20 2  0                                                        
           09:30:30 2  0                                                        
           09:30:40 2  0                                                        
           09:30:50 2  0                                                        
           09:31:00 2  0                                                        
           09:31:10 2  0                                                        
           09:31:20 2  0                                                        
           09:31:30 2  0                                                        
2019.01.03 09:31:40 2  -1761

In [7]:
/ reverse fills the missing values
tbl: update reverse fills reverse date,
 reverse fills reverse mid,
 reverse fills reverse spread,
 reverse fills reverse vol,
 reverse fills reverse adv 
 by id from tbl

In [8]:
select from tbl where id = 2

date       time     id trade     mid      spread       vol          adv         
--------------------------------------------------------------------------------
2019.01.03 09:30:00 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:30:10 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:30:20 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:30:30 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:30:40 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:30:50 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:31:00 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:31:10 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:31:20 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:31:30 2  0         44.15    0.002856641  0.0004827488 1.38682e+007
2019.01.03 09:31:40 2  -1761

# Exericse 1.1

For a given halflife h, trade vector, adv and vol, implement a function that returns the impact state vector.

In [9]:
computeImpact : {[trade; adv; vol; h]
 / exponential decay constant per 10 seconds, 
 / i.e. \beta * \delta t in the formula above
 beta: (log 2) % 6 * h; 
 lambda: 1;
 dailyVol: vol * sqrt 6 * 60 * 6.5;
 / initialize impact to be zero
 / drop the first elemetn from ema in the output
 (neg count trade)#ema[beta; 0, lambda * dailyVol * trade % beta * adv]}; 

# Exercise 1.2

Let $h = 1, 5, 30, 60$ min be halflifes. Using the cross operator, create a table that duplicates each row for each halflife $h$.

In [10]:
crossTbl: ([]h: 1,5, 30, 60) cross tbl

# Exercise 1.3

For each halflife h and stock, compute the impact state vector across time. Be sure to scale the model using adv and vol.

In [11]:
tbl2: update I: computeImpact[trade; adv; vol; first h] by id, h from crossTbl

In [12]:
select last time, impact_bps: 10000 * last I by id, h from tbl2

id h | time     impact_bps 
-----| --------------------
0  1 | 16:00:00 -4.413128  
0  5 | 16:00:00 -5.681994  
0  30| 16:00:00 -6.331318  
0  60| 16:00:00 -6.749565  
1  1 | 16:00:00 -2.845881  
1  5 | 16:00:00 -0.01882457
1  30| 16:00:00 7.720802   
1  60| 16:00:00 15.46085   
2  1 | 16:00:00 -0.1812012 
2  5 | 16:00:00 0.5857752  
2  30| 16:00:00 1.983902   
2  60| 16:00:00 1.611871   
3  1 | 16:00:00 -0.821108  
3  5 | 16:00:00 2.434364   
3  30| 16:00:00 8.661006   
3  60| 16:00:00 10.63799   
4  1 | 16:00:00 -0.3023859 
4  5 | 16:00:00 -3.146469  
4  30| 16:00:00 -11.21468  
4  60| 16:00:00 -17.51732  
5  1 | 16:00:00 0.2837103  
5  5 | 16:00:00 -1.624567  
5  30| 16:00:00 -5.12655   
5  60| 16:00:00 -5.983349  
6  1 | 16:00:00 3.328199   
..


# Exercise 1.4

Let $\Delta t = 1, 15, 60$ min be prediction horizons. Using the cross operator, create a table that duplicates each row for each prediction horizon. For each prediction horizon, compute the corresponding returns $r = P_{t+\Delta t}/P_t − 1$ and differences in impact states $\Delta I = I_{t+\Delta t} − I_t$.

In [13]:
crossTbl2: ([]DeltaT:1,15,60) cross tbl2

tbl3: update rtn:(neg 1)+mid%(6*DeltaT) xprev mid, 
    DeltaI: I + neg (6*DeltaT) xprev I by DeltaT, h, id from crossTbl2

In [14]:
tbl3

DeltaT h date       time     id trade     mid      spread        vol         adv           I     ..
-------------------------------------------------------------------------------------------------..
1      1 2019.01.03 09:30:00 0  454071.8  91.715   0.001148717   0.000446367 1.122415e+008 8.7351..
1      1 2019.01.03 09:30:10 0  -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008 4.4204..
1      1 2019.01.03 09:30:20 0  -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008 3.9039..
1      1 2019.01.03 09:30:30 0  -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008 1.9354..
1      1 2019.01.03 09:30:40 0  -24705.54 91.60259 0.0008423923  0.000446367 1.122415e+008 1.2365..
1      1 2019.01.03 09:30:50 0  -92166.19 91.58385 0.0005871219  0.000446367 1.122415e+008 -6.793..
1      1 2019.01.03 09:31:00 0  -22823.01 91.56276 0.000663703   0.000446367 1.122415e+008 -1.039..
1      1 2019.01.03 09:31:10 0  -4940.182 91.59088 0.0005615949  0.000446367 1.122415e+008 -1.014..


# Exercise 1.5

For each stock, halflife, and prediction horizon, compute the covariance $E[rΔI]$ and the variance $E[(ΔI)2]$.

In [15]:
tbl4: update covI: avg rtn*DeltaI, 
             varI: avg DeltaI*DeltaI, 
             varrtn: avg rtn*rtn 
      by DeltaT, h, id from tbl3

In [16]:
tbl5: select corr: first covI % sqrt varI*varrtn 
      by id, DeltaT, h from tbl4

In [17]:
select id, R2: corr*corr from tbl5

id R2          
---------------
0  0.1972367   
0  0.2935163   
0  0.3247731   
0  0.3250819   
0  0.07062189  
0  0.2929868   
0  0.6639954   
0  0.6884585   
0  0.002120665 
0  0.05056666  
0  0.4693113   
0  0.5095557   
1  0.09484972  
1  0.2118119   
1  0.2011021   
1  0.1924029   
1  0.0007992855
1  0.01756812  
1  0.1030524   
1  0.1221316   
1  0.001754568 
1  0.04026826  
1  0.2118507   
1  0.2609247   
2  0.1054773   
..
